In [1]:
# from google.colab import drive

# drive.mount("/content/drive")

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
TRAIN_IMG_PATH = "../dataset/bootcamp/train/images"
TRAIN_MASK_PATH = "../dataset/bootcamp/train/annotations"
VAL_IMG_PATH = "../dataset/bootcamp/test/images"
VAL_MASK_PATH = "../dataset/bootcamp/test/annotations"

EDA_DS_PATH = "../dataset/bootcamp/train"
EDA_TRAIN_LABELS = os.path.join(EDA_DS_PATH, "train_labels.csv")
EDA_TRAIN_FILES = os.path.join(EDA_DS_PATH, "train_files.csv")
EDA_VAL_FILES = os.path.join(EDA_DS_PATH, "val_files.csv")

MODEL_PATH = "../models/FCN8s"

os.makedirs(MODEL_PATH, exist_ok=True)  # Ensure directory exist

In [4]:
files_df = pd.read_csv(EDA_TRAIN_FILES)
labels_df = pd.read_csv(EDA_TRAIN_LABELS)
val_files_df = pd.read_csv(EDA_VAL_FILES)

In [5]:
print(files_df.head())
print()
print(labels_df.head())
print()
train_files = files_df['filename'].to_numpy()
print(train_files[:5])
print()
val_files = val_files_df['filename'].to_numpy()
print(val_files[:5])

            filename
0  0001TP_006690.png
1  0001TP_006720.png
2  0001TP_006750.png
3  0001TP_006780.png
4  0001TP_006810.png

   class     label
0      0       sky
1      1  building
2      2      pole
3      3      road
4      4  sidewalk

['0001TP_006690.png' '0001TP_006720.png' '0001TP_006750.png'
 '0001TP_006780.png' '0001TP_006810.png']

['0016E5_07959.png' '0016E5_07961.png' '0016E5_07963.png'
 '0016E5_07965.png' '0016E5_07967.png']


In [6]:
import torch
import torch.nn as nn
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import matplotlib.pyplot as plt

## Dataset Loader

In [7]:
# refer to dataset.py
from dataset import CityScapes

In [8]:
train_datasets = CityScapes(TRAIN_IMG_PATH, TRAIN_MASK_PATH, train_files)
image, mask = train_datasets.__getitem__(0)
print(image)
print()
print(mask)
print()
print(image.shape)
print()
print(mask.shape)

val_datasets = CityScapes(VAL_IMG_PATH, VAL_MASK_PATH, val_files)

tensor([[[-1.8439, -1.9124, -1.8782,  ..., -1.8268, -1.8610, -1.8268],
         [-1.8782, -1.8782, -1.8953,  ..., -1.8268, -1.8610, -1.8268],
         [-1.8953, -1.8610, -1.8610,  ..., -1.9124, -1.8782, -1.8439],
         ...,
         [-1.8610, -1.8610, -1.8439,  ..., -1.7240, -1.6727, -1.6727],
         [-1.8439, -1.8439, -1.8439,  ..., -1.6898, -1.6213, -1.7240],
         [-1.8439, -1.8610, -1.8439,  ..., -1.7240, -1.7240, -1.7412]],

        [[-1.7556, -1.8256, -1.7906,  ..., -1.6681, -1.7031, -1.7206],
         [-1.7906, -1.7906, -1.8081,  ..., -1.6681, -1.7031, -1.7206],
         [-1.8081, -1.7731, -1.7731,  ..., -1.7556, -1.7206, -1.7381],
         ...,
         [-1.7556, -1.7556, -1.7381,  ..., -1.4580, -1.4055, -1.3880],
         [-1.7381, -1.7381, -1.7381,  ..., -1.4580, -1.3704, -1.4405],
         [-1.7556, -1.7556, -1.7381,  ..., -1.4580, -1.4405, -1.4580]],

        [[-1.5256, -1.5953, -1.5604,  ..., -1.4210, -1.4559, -1.4384],
         [-1.5604, -1.5604, -1.5779,  ..., -1

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_datasets, batch_size=32, shuffle=True, num_workers=0)

## Model Architecture

In [10]:
from architecture import DoubleConv
import torch.nn as nn

class FCN8s(nn.Module):
  def __init__(self, in_channels, out_channels, features= [64, 128, 256, 512, 1024]):
    super().__init__()
    self.layers = nn.ModuleList()
    self.pool = nn.MaxPool2d(2, 2)
    self.out_channels = out_channels

    for feature in features:
      self.layers.append(DoubleConv(in_channels, feature))
      in_channels = feature

    self.ups1 = nn.ConvTranspose2d(features[-1], features[-2], kernel_size=2, stride=2)
    self.ups2 = nn.ConvTranspose2d(features[-1], features[-3], kernel_size=2, stride=2)

    self.predictions = nn.ConvTranspose2d(features[-2], out_channels, kernel_size=8, stride=8)

  def forward(self, x):
    skip_connections=[]

    for idx,layer in enumerate(self.layers):
      x = layer(x)
      x = self.pool(x)
      if idx in [2,3]:
        skip_connections.append(x)


    ups1 = self.ups1(x)
    concat1 = torch.concat([ups1, skip_connections[-1]], dim=1)

    ups2 = self.ups2(concat1)
    concat2 = torch.concat([ups2, skip_connections[-2]], dim=1)

    return self.predictions(concat2)

## Engine

In [11]:
from engine import DiceLoss
from engine import DiceCrossEntropyLoss
from engine import train

Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\notebooks\engine.py:137: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  def train_engine(dataloader, model, loss_fn, optim, scaler=torch.cuda.amp.GradScaler()):


In [12]:
model = FCN8s(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = DiceLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [13]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_dice.pth', last_model_name='last_model_dice.pth', save_path=MODEL_PATH)

100%|██████████████████████████████████████████████████████████████████████| 12/12 [00:12<00:00,  1.02s/it, loss=0.766]
Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\notebooks\engine.py:183: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.41it/s, loss=0.929]


Epoch 0/99
----------
Train Loss: 0.8564, Val Loss: 0.9288
✅ Best model saved! (Val Loss: 0.928842693567276)



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.46it/s, loss=0.883]


Epoch 1/99
----------
Train Loss: 0.7546, Val Loss: 0.8801
✅ Best model saved! (Val Loss: 0.8800560683012009)



100%|██████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.51it/s, loss=0.8]


...

Epoch 97/99
----------
Train Loss: 0.5386, Val Loss: 0.7104



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.39it/s, loss=0.709]


Epoch 98/99
----------
Train Loss: 0.5345, Val Loss: 0.7118



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.46it/s, loss=0.715]

Epoch 99/99
----------
Train Loss: 0.5341, Val Loss: 0.7146



In [14]:
import torch.nn as nn

model = FCN8s(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [ ]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_cross.pth', last_model_name='last_model_cross.pth', save_path=MODEL_PATH)

100%|███████████████████████████████████████████████████████████████████████| 12/12 [00:08<00:00,  1.34it/s, loss=2.27]
Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\notebooks\engine.py:183: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.44it/s, loss=3.96]


Epoch 0/99
----------
Train Loss: 2.1823, Val Loss: 3.9859
✅ Best model saved! (Val Loss: 3.9859291315078735)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.57it/s, loss=2.57]


Epoch 1/99
----------
Train Loss: 2.3264, Val Loss: 2.5777
✅ Best model saved! (Val Loss: 2.5777209997177124)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.63it/s, loss=2.42]


...

100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.53it/s, loss=1.26]


Epoch 31/99
----------
Train Loss: 1.0830, Val Loss: 1.2632
✅ Best model saved! (Val Loss: 1.2632010281085968)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.61it/s, loss=1.27]


Epoch 32/99
----------
Train Loss: 1.0695, Val Loss: 1.2561
✅ Best model saved! (Val Loss: 1.2560709714889526)



 83%|███████████████████████████████████████████████████████████▏           | 10/12 [00:07<00:01,  1.34it/s, loss=1.05]